In [ ]:
#Define the path to the directory where the data should be stored, as well as a unique identifier for each run
directory = "/data/s1968653/Vanilla_output/"
run_number = 17

In [ ]:
#Here we import all the necessary dependencies
import numpy as np

from amuse.ext.orbital_elements import new_binary_from_orbital_elements, get_orbital_elements_from_binary
from amuse.ext.solarsystem import new_solar_system
from amuse.lab import units, constants, Particles, nbody_system
from amuse.io import write_set_to_file
from amuse.community.mercury.interface import MercuryWayWard

from tqdm import tqdm

In [ ]:
#Takes a primary- and a secondary-particle and then returns the orbital parameters of their orbit. This function then returns
#the semi major axis. 

def sma_determinator(primary, secondary):
    binary = Particles(0)
    binary.add_particle(primary)
    binary.add_particle(secondary)
        
    orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
    semi_major_axis = orbital_params[2]
    return semi_major_axis

In [ ]:
#Function to generate orbits for comets in the Solar System.
def comet_positions_and_velocities(N_objects, sun_location):  
    positions = np.zeros((N_objects, 3)) | units.AU
    velocities = np.zeros((N_objects,3)) | units.kms
    
    m_sun = 1 | units.MSun
    m_comet = 0 | units.MSun #Take massless test particles
    for i in range(N_objects):
        # Values below correspond with random locations anywhere in the Solar System, based of relevant literature
        a = np.random.uniform(4, 40) | units.AU  # semi-major axis
        e = np.random.uniform(0, 0.05)  # eccentricity
        inclination = np.random.uniform(-5, 5) | units.deg
        true_anomaly = np.random.uniform (0, 360) | units.deg
        arg_of_periapsis = np.random.uniform(0, 360) | units.deg
        long_of_ascending_node = np.random.uniform(0, 360) | units.deg
        
        sun_and_comet = new_binary_from_orbital_elements(m_sun, m_comet, 
                                          a, e, true_anomaly, inclination, long_of_ascending_node, arg_of_periapsis, G=constants.G)
        positions[i] = (sun_and_comet[1].x+sun_location[0]), (sun_and_comet[1].y+sun_location[1]), (sun_and_comet[1].z+sun_location[2])
        velocities[i]= sun_and_comet[1].vx, sun_and_comet[1].vy, sun_and_comet[1].vz
    return positions, velocities

In [ ]:
#Function to create a post-tack system, i.e. with the giants at orbits as they are predicted to have been after the Grand Tack
def create_post_tack_giants_system():
    #Create the present day solar system and keep only the sun and the giants
    present_day_solar_system = new_solar_system()
    present_day_solar_system = present_day_solar_system[present_day_solar_system.mass > 10**-5 | units.MSun] # Takes gas giants and Sun only
    present_day_solar_system.move_to_center()
    
    #Create a post_tack_giants_system by first recreating the sun.
    post_tack_giants_system = Particles(1) 
    post_tack_giants_system[0].name = "Sun"
    post_tack_giants_system[0].mass = 1.0 | units.MSun
    post_tack_giants_system[0].radius = 1.0 | units.RSun  
    post_tack_giants_system[0].position = (0, 0, 0) | units.AU
    post_tack_giants_system[0].velocity = (0, 0, 0) | units.kms
    post_tack_giants_system[0].density = 3*post_tack_giants_system.mass/(4*np.pi*post_tack_giants_system.radius**3)
    
    #The post tack orbital elements for the planets as below
    a =  np.array([5.4, 7.1, 10.5, 13]) | units.AU 
    true_anomalies = np.random.uniform(0, 360, 4) | units.deg
    long_of_ascending_node = np.random.uniform(0, 360, 4) | units.deg
    args_of_periapsis = np.random.uniform(0, 360, 4) | units.deg
    
    #Create the four planets as binaries with the sun and add them to the post_tack_giants_system
    for i in range(4):
        orbital_elements = get_orbital_elements_from_binary(present_day_solar_system[0]+ present_day_solar_system[i+1], G=constants.G)
        inclination = orbital_elements[5] #Make sure we have a sensable inclination for the giants
        
        
        sun_and_planet = new_binary_from_orbital_elements(post_tack_giants_system.mass[0], present_day_solar_system[i+1].mass, 
                                          a[i], 0, true_anomalies[i], inclination, long_of_ascending_node[i], args_of_periapsis[i], G=constants.G)
        
        planet = Particles(1)
        planet.name = present_day_solar_system[i+1].name
        planet.mass = present_day_solar_system[i+1].mass
        planet.radius = present_day_solar_system[i+1].radius
        planet.position = (sun_and_planet[1].x-sun_and_planet[0].x, sun_and_planet[1].y-sun_and_planet[0].y, sun_and_planet[1].z-sun_and_planet[0].z)
        planet.velocity = (sun_and_planet[1].vx-sun_and_planet[0].vx, sun_and_planet[1].vy-sun_and_planet[0].vy, sun_and_planet[1].vz-sun_and_planet[0].vz)
        planet.density = 3*planet.mass/(4*np.pi*planet.radius**3) #MercuryWayWard requires a density parameter, so we define it here
        post_tack_giants_system.add_particle(planet) 
        
    return post_tack_giants_system
        
post_tack_giants_system = create_post_tack_giants_system()
post_tack_giants_system.move_to_center()

In [ ]:
#Define the number of comets and create their velocities and positions
N_objects = 1*10**2
sun_location = [post_tack_giants_system[0].x.in_(units.AU), post_tack_giants_system[0].y.in_(units.AU), post_tack_giants_system[0].z.in_(units.AU)]
comet_positions, comet_velocities = comet_positions_and_velocities(N_objects, sun_location)

In [ ]:
# Here we add the comets, where orbit parameters were chosen from a uniform distribution
def add_comet_objects(post_tack_giants_system, N_objects, comet_positions, comet_velocities):
    for i in tqdm(range(N_objects)):
        comet = Particles(1)
        comet.name = "COMET_" + str(i)
        comet.mass = 0.0 | units.MSun #Take massless test particles
        comet.radius = 0.0 | units.RSun 
        comet.position = (comet_positions[i, 0], comet_positions[i, 1], comet_positions[i, 2])
        comet.velocity = (comet_velocities[i, 0], comet_velocities[i, 1], comet_velocities[i, 2])

        post_tack_giants_system.add_particle(comet)
        
    for particle in post_tack_giants_system: #MercuryWayWard requires a density parameter, so we define it here
        particle.density = 3*particle.mass/(4*np.pi*particle.radius**3)
    
    return post_tack_giants_system

complete_post_tack_system = add_comet_objects(post_tack_giants_system, N_objects, comet_positions, comet_velocities)
complete_post_tack_system.move_to_center()

In [ ]:
#Here we create the converter
converter_length = get_orbital_elements_from_binary(complete_post_tack_system[0:2], G = constants.G)[2].in_(units.AU) # Typical distance used for calculation (=distance from Sun to Jupiter)
converter=nbody_system.nbody_to_si(complete_post_tack_system.mass.sum(), 
                                   converter_length)

In [ ]:
#Here we evolve the complete_post_tack_system, without a grandtack happening or a Milky way potential being present

def vanilla_evolver(complete_post_tack_system, converter, N_objects, end_time, time_step):
    #Initialise the gravity code and add the particles to it
    gravity_code = MercuryWayWard(converter)
    gravity_code.initialize_code()
    
    gravity_code.central_particle.add_particle(complete_post_tack_system[0]) #The sun is the central particle
    gravity_code.orbiters.add_particles(complete_post_tack_system[1:]) #Planets and comets are the orbiters
    gravity_code.commit_particles
    
    channel = gravity_code.particles.new_channel_to(complete_post_tack_system)
    
    times = np.arange(0., end_time, time_step) | units.yr #All time steps to which we want to evolve the model
    
    #---------------------------------------------------------------------------------------------------------
    #Here we define the planetary orbital parameters that should be returned to if the planets start moving too much 
    current_sma = np.array([0, 0, 0, 0]) | units.AU
    correct_sma =  np.array([5.4, 7.1, 10.5, 13]) | units.AU
    inclinations =  np.array([0, 0, 0, 0]) | units.deg
    
    
    system = new_solar_system()
    system = system[system.mass > 10**-5 | units.MSun] # Takes gas giants and Sun only
    system.move_to_center()
    for k in range(4):
        orbital_elements = get_orbital_elements_from_binary(system[0]+ system[k+1], G=constants.G)
        inclinations[k] =  orbital_elements[5]
    #------------------------------------------------------------------------------------------------------------
    dead_comets = [] #Here all 'dead' comets are stored
    
    #Below the evolving starts
    for i in tqdm(range(len(times))):
        gravity_code.evolve_model(times[i])
        channel.copy()
        
        #---------------------------------------------------------------------------------------------------------------
        #Here we check if the planetary orbits are still 'correct' and act for three degrees of incorrectness.
        for j in range(4):
            current_sma[j] = sma_determinator(gravity_code.central_particle, gravity_code.orbiters[j])
        
        for l in range(4):
            if abs(current_sma[l]/correct_sma[l]) > 1.25 or abs(current_sma[l]/correct_sma[l]) < 0.75: #The orbits are too much perturbed, so we end the simulation
                return
        
            elif abs(current_sma[l]/correct_sma[l]) > 1.05 or abs(current_sma[l]/correct_sma[l]) < 0.95: #The orbits are slightly perturbed, so we redefinie them
                print("Here", complete_post_tack_system[l+1].name, "was redefined")
                binary = Particles(0)
                binary.add_particle(gravity_code.central_particle)
                binary.add_particle(gravity_code.orbiters[l])

                orbital_params = get_orbital_elements_from_binary(binary, G = constants.G)
                true_anomaly, ascending_node, pericenter = orbital_params[4].in_(units.deg), orbital_params[6].in_(units.deg), orbital_params[7].in_(units.deg)

                sun_and_plan = new_binary_from_orbital_elements(1 | units.MSun, orbital_params[1], #We keep the current angles, but change the a, e and i back
                                                      correct_sma[l], 0, true_anomaly, inclinations[l], ascending_node, pericenter, G=constants.G)

                gravity_code.particles[l+1].position = (sun_and_plan[1].x-sun_and_plan[0].x, sun_and_plan[1].y-sun_and_plan[0].y, sun_and_plan[1].z-sun_and_plan[0].z)
                gravity_code.particles[l+1].velocity = (sun_and_plan[1].vx-sun_and_plan[0].vx, sun_and_plan[1].vy-sun_and_plan[0].vy, sun_and_plan[1].vz-sun_and_plan[0].vz)
            else: #The orbits do not need changing
                pass
        #----------------------------------------------------------------------------------------------------------------------
        #Once we checked for the orbital correctness, we can save data 
        
        if i%1000 == 0:
            write_set_to_file(gravity_code.orbiters, directory + 'Vanilla_run' + str(run_number) +'_time=' + str(np.log10(times[i].value_in(units.yr)))[0:5] + '.hdf5', format='hdf5', overwrite_file = True)
        
        #--------------------------------------------------------------------------------------------------------------------
        #Here we look for 'escaped' and 'out of bounds' comets
        out_of_bounds, escaped_comets = [], []
        for i in range(len(gravity_code.orbiters)):
            if gravity_code.orbiters[i].position.length() > 500 | units.AU:
                escaped_comets.append(gravity_code.orbiters[i].name)
                if gravity_code.orbiters[i].position.length() > 250000 | units.AU:
                    out_of_bounds.append(gravity_code.orbiters[i])
                    dead_comets.append(gravity_code.orbiters[i])
        for particle in out_of_bounds: #Out of bounds comets are removed completely
            complete_post_tack_system.remove_particle(particle)
            complete_post_tack_system.synchronize_to(gravity_code.particles)
            
        print("The amount of currently escaped comets is ", len(escaped_comets))
        print("The amount of dead comets is ", len(dead_comets))
        print("The planetary positions are ", gravity_code.orbiters[0].position.length().in_(units.AU), gravity_code.orbiters[1].position.length().in_(units.AU), gravity_code.orbiters[2].position.length().in_(units.AU), gravity_code.orbiters[3].position.length().in_(units.AU))
    
    gravity_code.stop()
    write_set_to_file(gravity_code.orbiters, directory + 'Vanilla_run' + str(run_number) +'_final.hdf5', format='hdf5', overwrite_file = True)
    return complete_post_tack
    
    
vanilla_evolved_system = vanilla_evolver(complete_post_tack_system, converter, N_objects, end_time= 10**8, time_step= 5*10**3)